# Convolutional Neural Networks for Visual Recognition
http://cs231n.github.io

## Image Classification

### Nearest Neighbor Classifier

** $L_1$ distance**

$d_1(I_1,I_2) = \sum_{p}|I_1^p-I_2^p|$

where $I_1$, $I_2$ are vectors of two images being compred.

**$L_2$ distance**

$d_2(I_1,I_2) = \sqrt{\sum_{p}(I_1^p-I_2^p)^2}$

**k-Nearest Neighbor Classifier**

Motivation: instead of finding single closest imgae in the traing set, we find the top k closest images and have them vote on the label of the test image.


![alt text](./img/k-nearest.png "Title")

### Pros and Cons of Nearest Neighbor Classifer
**Pros**
* Simple
* Takes no time to train

**Cons**
* May work on low-dimensional data but distance over high-dimensional spaces can be ver counter-intuitive
* Images that are nearby each other are much more a funciton of the general color distribution of the images, or the type of background rather than their semantic identity.
* The classifier must remember all the training data and store for future comparisons with the test data.
* Classifying a test image is expensive since it requires a comparison to all training images.


## Validation sets for Hyperparameter tuning

**Example of hyperparameters:** the setting for k of k-nearest neighbor classifier.

**Tuning hyperparameters:** split training test in two: a slightly smaller training set and what we call a validation set.

In case where the size of training data might be small, people sometimes use amore ophisticated techniques for hyperparameter tuning called **cross-validation**.

![alt text](./img/k-fold.png "Title")

Classic way is to split your training data randomly into train/val splits. As a rule of thumb, between 70-90% of your data usually goes to the train split. 

## Linear Classification
